In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

from mlp_models import VAE, CBM, OSR_VAE
from test_functions import *

import os

## LOAD VAE AND CBM
path = '../mnist_data/only_4_5/'

## INITIALIZATION
bs = 100
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='../mnist_data', train=True, transform=transform, download=True)
#train_dataset, val_dataset = torch.utils.data.random_split(dataset, [40000, 20000])

test_dataset = datasets.MNIST(root='../mnist_data', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
val_loader = train_loader #torch.utils.data.DataLoader(dataset=val_dataset, batch_size=bs, shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)


cbm = torch.load(path+'cbm.pt', map_location=torch.device('cpu'))
vae = torch.load(path+'vae.pt', map_location=torch.device('cpu'))
osr_vae = torch.load(path+'osr_vae_big.pt', map_location=torch.device('cpu'))

In [ ]:
from sklearn.linear_model import LogisticRegression


def log_regression(values, y, test_x, test_y, name='VAE'):
    z = torch.sigmoid(values)[:,:2].detach().numpy()
    y = y.detach().numpy().ravel()
    y = np.array(y%2==0, dtype=int)
    test_x = torch.sigmoid(test_x)[:,:2].detach().numpy()
    test_y = test_y.detach().numpy().ravel()
    test_y = np.array(test_y%2==0, dtype=int)
    
    print(len(z))
    
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(z,y)
    print(name+' Score:', log_reg.score(test_x,test_y)*100)
    print('Coeff:', log_reg.coef_)
    return log_reg

In [ ]:
import numpy as np

# LAOD DATASET

values_cbm  = torch.zeros(len(train_dataset), 2)
values_vae  = torch.zeros(len(train_dataset), 2)
values_osr_vae  = torch.zeros(len(train_dataset), 2)

targets = torch.zeros(len(train_dataset), 1)
final_mask = torch.zeros(len(train_dataset), dtype=torch.bool)
j = 0

bs = train_loader.batch_size
for data in train_loader:
    x, y = data
    mask = torch.tensor( (y != 4) & (y != 5), dtype=torch.bool)
    mu_cbm = cbm(x)
    recon, mu, log_var, z_vae = vae(x)
    recon, mu, log_var, z_osr = osr_vae(x)
    
    values_cbm[j*bs:(j+1)*bs] = mu_cbm[:,:2]
    values_vae[j*bs:(j+1)*bs] = z_vae[:,:2]
    values_osr_vae[j*bs:(j+1)*bs] = z_osr[:,:2]
    targets[j*bs:(j+1)*bs] = y.view(-1,1)
    final_mask[j*bs:(j+1)*bs] = mask
    j += 1

values_cbm = values_cbm[final_mask]
values_vae = values_vae[final_mask]
values_osr_vae = values_osr_vae[final_mask]
targets = targets[final_mask]

#########################################################################

test_values_cbm  = torch.zeros(len(test_dataset), 2)
test_values_vae  = torch.zeros(len(test_dataset), 2)
test_values_osr_vae  = torch.zeros(len(test_dataset), 2)

test_targets = torch.zeros(len(test_dataset), 1)
test_final_mask = torch.zeros(len(test_dataset), dtype=torch.bool)
j = 0
bs = test_loader.batch_size
for data in test_loader:
    x, y = data
    mask = torch.tensor( (y != 4) & (y != 5), dtype=torch.bool)
    mu_cbm = cbm(x)
    _, _, _, z_vae = vae(x)
    _, _, _, z_osr = osr_vae(x)

    test_values_cbm[j*bs:(j+1)*bs] = mu_cbm[:,:2]
    test_values_vae[j*bs:(j+1)*bs] = z_vae[:,:2]
    test_values_osr_vae[j*bs:(j+1)*bs] = z_osr[:,:2]
    test_targets[j*bs:(j+1)*bs] = y.view(-1,1)
    test_final_mask[j*bs:(j+1)*bs] = mask
    j += 1
test_values_cbm = test_values_cbm[test_final_mask]
test_values_vae = test_values_vae[test_final_mask]
test_values_osr_vae = test_values_osr_vae[test_final_mask]
test_targets = test_targets[test_final_mask]

In [ ]:
# representation of 4 and 5 in the latent space

train_values_cbm  = torch.zeros(len(train_dataset), 2)
train_values_vae  = torch.zeros(len(train_dataset), 2)
train_values_osr  = torch.zeros(len(train_dataset), 2)

train_targets = torch.zeros(len(train_dataset), 1)
train_mask = torch.zeros(len(train_dataset), dtype=torch.bool)
j = 0

bs = train_loader.batch_size
for data in train_loader:
    x, y = data
    mask = torch.tensor( (y == 4) | (y == 5), dtype=torch.bool)        
    
    mu_cbm = cbm(x)
    recon, mu, log_var, z_vae = vae(x)
    recon, mu, log_var, z_osr = osr_vae(x)

    train_values_cbm[j*bs:(j+1)*bs] = mu_cbm[:,:2]        
    train_values_vae[j*bs:(j+1)*bs] = z_vae[:,:2]
    train_values_osr[j*bs:(j+1)*bs] = z_osr[:,:2]

    train_targets[j*bs:(j+1)*bs] = y.view(-1,1)
    train_mask[j*bs:(j+1)*bs] = mask
    j += 1
train_values_cbm = train_values_cbm[train_mask]
train_values_vae = train_values_vae[train_mask]
train_values_osr = train_values_osr[train_mask]
train_targets = train_targets[train_mask]

In [ ]:
### OPENSET MECHANISM - DIFFIDENT ####

In [ ]:
import scipy.integrate as integrate
from scipy.stats import norm

In [ ]:
y4 = torch.tensor([1,0], dtype=torch.float) 
y5 = torch.tensor([0,1], dtype=torch.float) 
mu4 = osr_vae.z_enc(y4).detach().numpy()
mu5 = osr_vae.z_enc(y5).detach().numpy()

In [ ]:
## CREATE Z in 4 and 5
values_osr_vae  = torch.zeros(len(train_dataset), 2)

targets = torch.zeros(len(train_dataset), 1)
rec_error =torch.tensor(0.)
final_mask = torch.zeros(len(train_dataset), dtype=torch.bool)
j = 0

bs = train_loader.batch_size
rec_errors = torch.zeros(len(train_dataset))
print(rec_errors.size())
for data in (train_loader):
    x, y = data
    mask = torch.tensor( (y == 4) | (y == 5), dtype=torch.bool)
    recon, mu, log_var, z = osr_vae(x)
    
    r = F.binary_cross_entropy(recon, x.view(-1, 784), reduction='mean')
    rec_error += r
    for k in range(bs):
        rec_errors[bs*j+k] = F.binary_cross_entropy(recon[k].view(-1, 784), x[k].view(-1, 784), reduction='mean') 
    #print(r)
    values_osr_vae[j*bs:(j+1)*bs] = z[:,:2]
    targets[j*bs:(j+1)*bs] = y.view(-1,1)
    final_mask[j*bs:(j+1)*bs] = mask
    j += 1
    
values_osr_vae = values_osr_vae[final_mask]
targets = targets[final_mask]
rec_error /= j

rec_thr = rec_error.detach().numpy()

In [ ]:
## EVALUATE REC THR

recons = rec_errors[final_mask].detach().numpy()
l = len(recons)
r_min, r_max = np.min(recons), np.max(recons)
good_r = []
for eta in np.linspace(r_min, r_max, 1000):
    mask = (recons < eta)
    if len( recons[mask] )/l > 0.94 and len( recons[mask] )/l < 0.96:
        good_r.append(eta)
rec_thr = np.mean(good_r)

In [ ]:
## CALCULATE PROB OF INTEGRALs
import scipy
from scipy.stats import norm
import scipy.integrate as integrate

zs = values_osr_vae.detach().numpy()
y = targets.detach().numpy().squeeze()

In [ ]:
# FIND eta_4 and eta_5

# ETA 4
good_dist = []

mask = (y%2 == 0)
l = len(zs[mask])

print(l)

dist = (mu4 - zs[mask])
dist = np.array([np.linalg.norm(d) for d in dist ])
print(dist)
for eta in np.linspace(0, 2, 1000):
    
    conds = dist < eta 
    
    if conds.sum()/l > 0.94 and  conds.sum()/l < 0.96:
        good_dist.append(eta)

eta4 = np.mean(good_dist)
 

# ETA 5
    
good_dist = []

mask = (y%2 == 1)
l = len(zs[mask])

print(l)

dist = (mu5 - zs[mask])
dist = np.array([np.linalg.norm(d) for d in dist ])
print(dist)
for eta in np.linspace(0, 2, 1000):
    
    conds = dist < eta 
    
    if conds.sum()/l > 0.94 and  conds.sum()/l < 0.96:
        good_dist.append(eta)

eta5 = np.mean(good_dist)
 

In [ ]:
### NOW WE HAVEBOTH THR, let's see on new data what happens

In [ ]:
import numpy as np
import pandas as pd

values_osr_vae  = torch.zeros(len(train_dataset), 2)
targets = torch.zeros(len(train_dataset), 1)
final_mask = torch.zeros(len(train_dataset), dtype=torch.bool)
accepted = torch.zeros(len(train_dataset), dtype=torch.bool)

print('Dataset:', len(train_dataset))
j = 0

bs = train_loader.batch_size
for data in train_loader:
    x, y = data
    mask = torch.tensor( (y != 4) & (y != 5), dtype=torch.bool)
    recon, mu, log_var, z = osr_vae(x)
    for i in range(len(recon)):
        r = F.binary_cross_entropy(recon[i].view(-1, 784), x[i].view(-1, 784), reduction='mean')
        #print(r)
        if r.item() <= rec_thr:
            accepted[j*bs+i] = True
        
    values_osr_vae[j*bs:(j+1)*bs] = z[:,:2]
    targets[j*bs:(j+1)*bs] = y.view(-1,1)
    final_mask[j*bs:(j+1)*bs] = mask
    j += 1
    
values_osr_vae = values_osr_vae[final_mask]
targets = targets[final_mask]
accepted = accepted[final_mask]

In [ ]:
# CREATE TRAIN DATASET for LEAKAGE EXP

not_4_5_train = pd.DataFrame()
not_4_5_train['z1'] = values_osr_vae[:,0].detach().numpy()
not_4_5_train['z2'] = values_osr_vae[:,1].detach().numpy()
not_4_5_train['targets'] = targets.detach().numpy()
not_4_5_train['rec_accepted'] = accepted.detach().numpy()
not_4_5_train.head()

In [ ]:
mask = np.array(not_4_5_train['rec_accepted'])
l = len(not_4_5_train)
d = len(targets[mask])

In [ ]:
test_values_osr_vae  = torch.zeros(len(test_dataset), 2)

test_targets = torch.zeros(len(test_dataset), 1)
test_final_mask = torch.zeros(len(test_dataset), dtype=torch.bool)
accepted = torch.zeros(len(test_dataset), dtype=torch.bool)

j = 0
bs = test_loader.batch_size
for data in test_loader:
    x, y = data
    mask = torch.tensor( (y != 4) & (y != 5), dtype=torch.bool)
    _, _, _, z_osr = osr_vae(x)
    
    for i in range(len(recon)):
        r = F.binary_cross_entropy(recon[i].view(-1, 784), x[i].view(-1, 784), reduction='mean')
        #print(r)
        if r.item() <= rec_thr:
            accepted[j*bs+i] = True
    
    test_values_osr_vae[j*bs:(j+1)*bs] = z_osr[:,:2]
    test_targets[j*bs:(j+1)*bs] = y.view(-1,1)
    test_final_mask[j*bs:(j+1)*bs] = mask
    j += 1
    
test_values_osr_vae = test_values_osr_vae[test_final_mask]
test_targets = test_targets[test_final_mask]
accepted = accepted[test_final_mask]

In [ ]:
# CREATE TEST DATASET for LEAKAGE EXP

not_4_5_test = pd.DataFrame()
not_4_5_test['z1'] = test_values_osr_vae[:,0].detach().numpy()
not_4_5_test['z2'] = test_values_osr_vae[:,1].detach().numpy()
not_4_5_test['targets'] = test_targets.detach().numpy()
not_4_5_test['rec_accepted'] = accepted.detach().numpy()
not_4_5_test.head()

In [ ]:
## see acceptance of train wrt z integral

data = np.asarray(not_4_5_train[['z1','z2']])
z_accepted = np.zeros(len(data), dtype=bool)
#var = np.mean(variances)
for i,z in enumerate(data):
    
    if np.linalg.norm(z - mu4) < eta4:
        z_accepted[i] = True
    elif np.linalg.norm(z - mu5) < eta5:
        z_accepted[i] = True

In [ ]:
not_4_5_train['z_accepted'] = z_accepted

In [ ]:
data = np.asarray(not_4_5_test[['z1','z2']])
z_accepted = np.zeros(len(data), dtype=bool)
for i,z in enumerate(data):
    if np.linalg.norm(z - mu4) < eta4:
        z_accepted[i] = True
    elif np.linalg.norm(z - mu5) < eta5:
        z_accepted[i] = True

not_4_5_test['z_accepted'] = z_accepted

In [ ]:
# INTRODUCE THE LOG REGRESSION WITH REJECTION

def diffident_log_regression(values, y, test_values, test_y, which_ones, name='VAE'):
    z = values
    y = np.array(y%2==0, dtype=int)
    test_x = test_values
    test_y = np.array(test_y%2==0, dtype=int)
    
    print(len(z))
    
    log_reg = LogisticRegression(max_iter=1000)
    log_reg.fit(z,y)
    
    accuracy = 0
    pred = np.zeros(len(test_x))
    for i in range(len(test_x)):
        if which_ones[i]:
            pred[i] = log_reg.predict(test_values[i].reshape(1,-1))
        else:
            pred[i] = 0.5
            test_x[i] = np.array([0,0])
        sort = np.random.binomial(1, pred[i], size=None)
        if test_y[i] == sort:
            accuracy += 1
    
    print(name+' Score: with acc', accuracy/len(which_ones))
    print('Score with log_reg:', log_reg.score(test_x, test_y) )
    print('params', log_reg.coef_)
    return log_reg

def diffident_prediction(log_reg, zs, which_ones):
    preds = []
    for i, z in enumerate(zs):
        if which_ones[i]:
            preds.append(log_reg.predict(z.reshape(1,-1)))
        else:
            preds.append(np.random.randint(2))
    return np.array(preds)

In [ ]:
which_ones = not_4_5_train['rec_accepted'] & not_4_5_train['z_accepted']
zs = not_4_5_train[['z1','z2']].values[which_ones]
y = not_4_5_train['targets'][which_ones] 

which_ones = not_4_5_test['rec_accepted'] & not_4_5_test['z_accepted']
test_zs = not_4_5_test[['z1','z2']].values
test_y = not_4_5_test['targets'] 

print(len(test_zs), len(test_y))
print(len(which_ones))

diff_log = diffident_log_regression(zs, y, test_zs, test_y, which_ones, name='OSR_VAE')

In [ ]:
preds = diffident_prediction(diff_log, test_zs, which_ones)